In [ ]:
import os  # <--- Add this line
from langgraph.graph import StateGraph ,START, END
from typing import TypedDict
from langchain_ollama import ChatOllama
from dotenv import load_dotenv

In [ ]:
load_dotenv()
ollama_url = os.getenv("OLLAMA_BASE_URL")
model = ChatOllama(
    model="llama3.2:latest", 
    base_url=ollama_url,  # This is the crucial part!
    temperature=0
)

In [ ]:
class blogState(TypedDict):
    topic:str
    outline:str  
    blog:str

In [ ]:
def generate_outline(state:blogState)->blogState:
    #fetch title
    topic=state['topic']    

    #form a prompt
    prompt=f'generate outline for topic {topic}'

    #call lmm for outline
    outline=model.invoke(prompt).content

    #updatestate
    state['outline']=outline

    return state

In [ ]:
def generate_blog(state:blogState)->blogState:
    #fetch title + outline
    topic=state['topic']  
    outline=state['blog']  

    #form a blog prompt
    prompt=f'generate detail blog for given topic and outline {topic , outline} '

    #call llm for blog
    blog=model.invoke(prompt).content

    #updating blog state
    state['blog']=blog
    
    return state

In [ ]:
#3.define graph(pass state)
graph=StateGraph(blogState)

#4.make node of function
graph.add_node('generate_outline'  , generate_outline)
graph.add_node('generate_blog'  , generate_blog)

#5.add edges between nodes
graph.add_edge(START,'generate_outline')
graph.add_edge('generate_blog','generate_outline')
graph.add_edge('generate_blog',END)

#6.complie workflow
workflow=graph.compile()

In [ ]:
#execute
inital_state={'topic':'Artificial intellegence'}
final_state = workflow.invoke(inital_state)
print(final_state)